# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [1]:
from urllib.request import urlopen

def stream(n):
    i = 0
    with urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [2]:
STREAM_SIZE = 10
web_traffic_stream = stream(STREAM_SIZE)
# list(web_traffic_stream)

## Data structures

Our data structure is based on the Flajolet-Martin algorithm, which the Hyperloglog algorithm is also based on.
To implement this algorithm, and estimate the number of unique IPs in the stream, we defined some helper functions: make_bucket, hash, and calc_prefix_length.
Essentially, we are trying to estimate the number of unique elements in the stream by hashing the IP address into a bit string.
We know that the random chance of having a 0 in the first position of the bit string is 1/2, thus we can assume that the probability will start with 'k' number of 0's is (1/2)^k. We then use 2^k as the estimate of the number of unique elements seen in the stream.
Additionally, to reduce the variance of overestimating, we introduce X amount of buckets, which are different hash functions in the same hash familiy (murmurhash3).
Each bucket represents the largest number of zeros seen from any element in the stream for the corresponding hash function.
Finally, we take the average of the bucket values (k) and use this to calculate our estimate (2^k).

To make the estimate better, it has been found in litterature (http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf?fbclid=IwAR3eFpcI7Tfg7IFS6T3RQO7eNjHQ1COgVo8hFnh7PTqPIM2K55hK4TFcoyg) that removing the top 30% of the bucket values will improve the results significantly. Therefore, this strategy was implemented, yielding us better estimates.


Estimating the X most frequent IPs in the stream:
The estimate was created by implementing the Count-min sketch. A 'd x w' matrix M was created, where d represents hash functions within the same family and w the number of possible hash positions.
Thus implementing stacked bloom filters containing individual counts of hashed 'IP + domain'-strings. We chose to hash the IPs and domains together in order to look at each IP's occurance in the stream. Additionally, the total occurance of an IP in the stream was determined by hashing IP + the string 'total' and incrementing the appropriate position in the M matrix.

The 'count_ip' function finds the estimated count of a specific IP in a particular domain or 'total', by looking up in the M matrix and taking the minimal value of the positions specified by the hash functions.

In [3]:
import mmh3
import numpy as np


# Function for making hashing more readable. Returning the absolute value of a 32-bit hashing.
def hash(element, seed):
    return abs(mmh3.hash(element, seed = seed))

# Function to calculate the length of a bit string prefix of 0s:
def calc_prefix_length(bit_string):
    # The initial 3 values in a bit string are not directly related to the actual number.
    bit_string = bit_string[3:]
    prefix_length = bit_string.find('1')
    return prefix_length


# Function for creating or editing a bucket of prefix lengths, based on hashing in different seed values (0-BUCKET_SIZE).
def make_bucket(element, bucket):
    for i in range(BUCKET_SIZE):
        hash_val = str(bin(hash(element, seed = i)))
        prefix_len = calc_prefix_length(hash_val)
        if prefix_len > bucket[i]:
            bucket[i] = prefix_len
    return bucket


# Function for estimating the count of a specific IP.
def count_ip(IP, domain):
    X_min = 10**10  # Some large initial value
    for i in range(d):
        # In order to distinguish IP and domain combinations we combine these in the hash function.
        wi = hash(IP+domain, seed = i) % w # w defines the size limit of returned hash values.
        X_val = M[i, wi]
        if X_val < X_min:
            X_min = X_val
    return X_min


In [127]:
%%time
# Initialize constants
BUCKET_SIZE = 32        # Number of hash functions in Flajolet-Martin algorithm
STREAM_SIZE = 100000    # Number of elements to go through
d = 10                  # Number of hash functions in Count-min sketch
w = STREAM_SIZE         # Number of possible hash values in Count-min sketch
M = np.zeros((d, w))    # Count matrix for Count-min sketch
X = 10                  # X IPs with highest occurance (estimate)
max_count = [0]*X       # Count of occurances of X IPs used for sorting the max_ip list
max_ip = [0]*X          # List of X most frequent IPs


domains = dict()        # Dictionary of domains with corresponding buckets
domains['total'] = [0]*BUCKET_SIZE


for element in stream(STREAM_SIZE):
    element = element.split('\t')
    IP = element[0]
    domain = element[1]
    
    # Add to total unique count
    domains['total'] = make_bucket(IP, domains['total'])
    # Add to domains dict according to domain 
    if domain in domains:
        domains[domain] = make_bucket(IP, domains[domain])
    else:
        domains[domain] = [0]*BUCKET_SIZE
    
    
    # Count-min sketch
    for i in range(d):
        wi = hash(IP+domain, i) % w     # Location of hashed IP with domain
        wit = hash(IP+'total', i) % w   # Location of hashed IP + 'total'
        M[i, wi] += 1
        M[i, wit] += 1
    
    # Estimating the X most frequent IPs
    if IP not in max_ip:
        num_ip = count_ip(IP, 'total')
        max_ip = [ip for count, ip in sorted(zip(max_count, max_ip))]
        max_count.sort()
        if num_ip > max_count[0]:
            max_count[0] = num_ip
            max_ip[0] = IP


CPU times: user 14.4 s, sys: 12 ms, total: 14.4 s
Wall time: 14.4 s


Short description of 'Estimating the X most frequent IPs':

* We only consider a IP that is not already there.
* 1) Estimate the count of a particular IP
* 2) Sort max_ip and max_count in order to easily compare with lowest value and insert if higher.
* 3) Add if it is higher than the total count of the current IP, or if there is not already an IP.

The needed data structures (domains, M, and max_ip) are now ready for our analysis.

## Analysis


#### How many unique IPs are there in the stream?
To answer this question, we define a function that exstracts the bucket of a specific domain from the domains dictionary.
As explained in the beginning, better results are yielded when removing the top 30% of the bucket. However, when using a harmonic mean the outliers have less influence and therefore we reduced this to 20%, partially because it gave better results.
The function returns 2^k, where k is the harmonic mean of the lowest 80% of the bucket.

In [132]:
def harmonic_mean(alist):
    n = len(alist)
    return n/np.sum([1/val for val in alist])

def get_count(domain):
    last_20 = round(len(domains[domain])*0.2)
    bucket = sorted(domains[domain])
    bucket = bucket[:-last_20]
    return int(round(2**harmonic_mean(bucket)))

In [133]:
print('The number of unique IPs are:', get_count('total'))

The number of unique IPs are: 93892


#### How many unique IPs are there for each domain?
We loop through the domains the domains dictionary and run the get_count function for each domain:

In [7]:
# How many unique IPs are there for each domain?
for domain in domains.keys():
    print('{} has {} unique IPs'.format(domain, get_count(domain)))

total has 88205 unique IPs
python.org has 33868 unique IPs
wikipedia.org has 35004 unique IPs
pandas.pydata.org has 7669 unique IPs
dtu.dk has 1680 unique IPs
google.com has 1680 unique IPs
databricks.com has 840 unique IPs
github.com has 868 unique IPs
spark.apache.org has 302 unique IPs
datarobot.com has 172 unique IPs
scala-lang.org has 1 unique IPs


#### How many times was IP X seen on domain Y? (for some X and Y provided at run time)
Here, the count_ip functions comes in handy, where we look up the IP + domain combination in the M Count-min matrix using the count_ip function.

In [8]:
# How many times was IP X seen on domain Y? (for some X and Y provided at run time)
# Define input variables:
IP_X = '72.187.84.158'
domain_Y = 'scala-lang.org'

count = count_ip(IP_X, domain_Y)
print('IP {} was seen {} times on {}'.format(IP_X, count, domain_Y))


IP 72.187.84.158 was seen 26.0 times on scala-lang.org


#### How many unique IPs are there for the domains d1,d2,…?
This is done in the same way as before, however, a list of domain names can be defined if desired.

In [9]:
# How many unique IPs are there for the domains d1,d2,…?
domain_names = ['python.org', 'wikipedia.org', 'pandas.pydata.org', 'github.com'] # domains.keys()

for domain in domain_names:
    print('The number of unique IPs visiting {} is: {}'.format(domain, get_count(domain)))


The number of unique IPs visiting python.org is: 33868
The number of unique IPs visiting wikipedia.org is: 35004
The number of unique IPs visiting pandas.pydata.org is: 7669
The number of unique IPs visiting github.com is: 868


#### How many times was IP X seen on domains d1,d2,…?
Again a list of domains of interest are defined and the tables are looped through, counting the number of times a specific IP occurs.

In [10]:
# How many times was IP X seen on domains d1,d2,…?
IP_X = '72.187.84.158'
domains_to_look_at = ['python.org', 'wikipedia.org', 'pandas.pydata.org', 'github.com', 'scala-lang.org'] # domains.keys()
for domain in domains_to_look_at:
    print(domain, count_ip(IP_X, domain))

python.org 0.0
wikipedia.org 0.0
pandas.pydata.org 0.0
github.com 0.0
scala-lang.org 26.0


#### What are the X most frequent IPs in the stream?
The X most frequent IPs were stored in the max_ip list. As the max_count was not reevaluated constantly, we need to use the count_ip function on the X IPs in the max_ip list:

In [11]:
print('Top', X, 'IPs and their frequency:')
for ip in max_ip:
    print(ip, count_ip(ip,'total'))

Top 10 IPs and their frequency:
56.29.201.127 5.0
56.30.200.129 5.0
57.30.198.127 5.0
204.141.72.187 7.0
54.28.199.128 6.0
55.29.200.127 6.0
55.30.200.128 6.0
56.29.201.129 6.0
108.41.112.108 27.0
72.187.84.158 26.0


##### Document the accuracy of your answers when using algorithms that give approximate answers
The Count-min sketch error analysis is derived from this article: http://dimacs.rutgers.edu/~graham/pubs/papers/cmencyc.pdf


In [139]:
import math
# Count-min sketch error analysis:
print('Error added with each element added to the M matrix:', math.e/w)
print('Probability of allowing a count estimate outside the error above:', math.exp(-d))


Error added with each element added to the M matrix: 2.7182818284590452e-05
Probability of allowing a count estimate outside the above error: 4.5399929762484854e-05


We can thus conclude that with the chosen parameters for d and w, we have a very high chance of hitting the true value, i.e. a high accuracy.

##### Argue why you are using certain parameters for your data structures

* **BUCKET_SIZE = 32**        -- We use 32 to minimize the risk of having a string hashing to a bit value with many initial 0s. With a higher bucket size, the chance of hitting critically wrong is diminished, as the harmonic mean of the buckets are used.
* **STREAM_SIZE = 100000**    -- A stream size of 100000 is used to run through a significant number of elements in the stream. 
* **d = 10**         -- For the Count-min sketch, the chance of two strings hashing to the same position reduces with a higher d and w. In our case, a d of 10 gave very good estimates when compared to actual counts.
* **w = STREAM_SIZE**         -- A w of 100000 means there are enough possible hash values for each single element in the stream. This combined with the d of 30 gives a total of 30 * 100000 positions in the M matrix and thus when using the Count-min sketch, a very good estimate of a count is achieved.